In [8]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from nltk import word_tokenize

/home/frfa/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/home/frfa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: htt

In [ ]:
# Loading embeddings

import gensim.models

googEmbs = gensim.models.KeyedVectors.load_word2vec_format(
                                '../embeddings/googlenews.bin', binary=True)
print('loading finished')

In [ ]:
### OLD Preprocess. Using torchtext instead ###

"""def sentence_embed(sentence): # Input sentence as string
    
    sentence = word_tokenize(sentence) # Preprocessing step: Tokenize
    
    emb_sent = []
    for word in sentence:
        try:
            emb_word = googEmbs[word]
        except KeyError:
            emb_word = np.zeros((300,))
        
        emb_sent.append(emb_word)
    
    return emb_sent # A list of word embeddings"""

# Defining the NN Model

GRU NN with last output through a sigmoid or a FNN with sigmoid to classify sentiment

In [ ]:
# Hyperparameters

input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 2
learning_rate = 0.001
batch_size = 64
num_epoch = 1


In [ ]:
# GRU RNN

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.fc = nn.Linear(hidden_size * sequence_length, num_classes) # All outputs of time t
        self.fc = nn.Linear(hidden_size, num_classes) # Only last output
        
    def foward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # Forward Prop
        out, _ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        #out = self.fc(out) # All outputs of time t
        out = self.fc(out[:, -1, :]) # Only last output
        
        return out
    

# Loading and preprocessing the data

Using torchtext: 
- https://torchtext.readthedocs.io/en/latest/index.html
- https://galhever.medium.com/sentiment-analysis-with-pytorch-part-1-data-preprocessing-a51c80cc15fb
- https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb

In [4]:
# Loading train data

from loader import load_train
train = load_train()

In [9]:
# Preprocessing data

from torchtext.legacy import data
import spacy 

#spacy.cli.download("en_core_web_sm")

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [21]:
# Formating train_x and train_y

train_x = list(map(TEXT.preprocess, train["reviewText"]))
train_x = TEXT.pad(train_x)
train_y = list(map(LABEL.preprocess, train["sentiment"]))

In [30]:
# Building vocab

TEXT.build_vocab(train_x, max_size=5000)
LABEL.build_vocab(train_x)
vocab_size = len(TEXT.vocab)